In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import wandb

import numpy as np
import seaborn as sns
import torch

import sys
BASE_PATH = globals()['_dh'][0].parent.absolute()
sys.path.insert(1, str(BASE_PATH))

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from src.utils.utils import make_Planetoid_data, exp_per_model, make_uniform_schedule
from src.utils.metrics import MAD
from src.models.iterativeModels import learnable_adaptive_iGCN, iterativeGCN
from src.models.GCN import GCN
import torch.nn.functional as F
wandb.login()

/Users/sizhuang/Desktop/GitHubRepos/IterativeMethods/iterENV/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/sizhuang/Desktop/GitHubRepos/IterativeMethods/iterENV/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
wandb: Currently logged in as: sizhuang. Use `wandb login --relogin` to force relogin


True

In [23]:
hyper = {
    'num_epochs': 200,
    'dataset_name': "Cora",
    'noise_percent': 0,
    'hid_dim': 32,
    'num_iter_layers': 12,
    'smooth_fac': 0.8,
    'dropout': 0.5,
    'learning_rate': 0.01,
    'weight_decay': 4e-4
} 
data, num_features, num_classes = make_Planetoid_data(hyper)

In [24]:
wandb.init(config=hyper, 
               job_type="over_smoothing", 
               project="IterativeMethods", 
               tags=["iterativeGCN"])
config = wandb.config
train_schedule = make_uniform_schedule(config.num_iter_layers, config.smooth_fac)
data, num_features, num_classes = make_Planetoid_data(config)
wandb.log({
    'train_schedule': train_schedule
})

model = iterativeGCN(input_dim=num_features,
                                output_dim=num_classes,
                                hidden_dim=config.hid_dim,
                                train_schedule=train_schedule,
                                dropout=config.dropout)
exp_per_model(model, data, config)

out = model(data.x, data.edge_index)
mad1 = MAD(out.detach())
mad2  =MAD(F.log_softmax(out, dim=1))
mad3 = MAD(F.softmax(out, dim=1))
// print(mad1)
// print(mad2)
print(mad3)

wandb.finish()

0.78588575
0.29897282
0.8116482


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
num_param,▁
test_accuracy,▁
test_loss,▁
training_accuracy,▁▁▁▂▂▃▅▅▆▆▆▆▇▇▇▆▇▇█▇▇██████████▇███████▇
training_loss,████▇▆▅▄▃▃▃▃▂▂▂▃▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
validation_accuracy,▁▁▂▃▃▄▅▅▇▇▇██▇████▇▇▇█▇█████████████▇███
validation_loss,▅▅▅▄▃▂▂▂▁▁▂▂▂▄▃▃▃▃▄▅▄▅▆▅▅▆▇▇▆▆▆▆▆▆▆██▇██
epoch,200
num_param,47175
test_accuracy,0.645


In [25]:
wandb.init(job_type="over_smoothing", 
               project="IterativeMethods", 
               config=hyper, 
               notes="usualGCN",
               tags=["usualGCN"])
config = wandb.config
model2 = GCN(input_dim=num_features,
                                output_dim=num_classes,
                                hidden_dim=config.hid_dim,
                                num_layers=config.num_iter_layers,
                                dropout=config.dropout,
                                )
exp_per_model(model2, data, config)
out2 = model2(data.x, data.edge_index)

mad1 = MAD(out2.detach())
mad2 = MAD(F.log_softmax(out2, dim=1))
mad3 = MAD(F.softmax(out2, dim=1))

# print(mad1)
# print(mad2)
print(mad3)

wandb.finish()

0.68794906
0.014618884
0.05222061


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
num_param,▁
test_accuracy,▁
test_loss,▁
training_accuracy,▁▃▅▂▂▁▅▄▂▆▅▂▄▁▁▂▃▃▃▁▄▁▆▅▆▂▅▆█▇▃▅▄▅▆▅▅▁▁▃
training_loss,█████▇▇▇▇▇█▇▇▇█▇▇█▇▇▇▇▆▅▅▅▄▄▂▃▄▆▄▃▃▁▄▄▄▄
validation_accuracy,▂▁▁▁▅▆▂▃▄▃▃█▄▂▄▃▅▅▅▄▆▄▅▄▄▆▅▃▆▄▅▅▅▅▄▄▅▆▅▆
validation_loss,▆▆▅▆▆▅▆▆▆▇█▅██▇▇▇▇▇▇▆▆▆▆▇▅▄▆▆▆▃▂▃▄▄▃▃▁▂▁
epoch,200
num_param,56679
test_accuracy,0.184


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
torch.set_printoptions(profile="full", sci_mode=False, precision=4)
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
out2 = model2(data.x, data.edge_index)
D = 1 - cosine_similarity(out2.detach())
print(D)

# other

In [ ]:
iMAD = [0.7474, 0.7881, 0.8062, 0.8044, 0.8058, 0.8207, 0.8070, 0.7913, 0.8005, 0.8042, 0.8061, 0.7774, 0.7152, 0.7403, 0.8081, 0.7029, 0.7716, 0.7167, 0.7368]
uMAD = [0.6512, 0.7687, 0.8147, 0.8125, 0.8088, 0.7588, 0.6406, 0.5322, 0.0153, 0.0537, 0.0821, 0.1540, 0.0249, 0.0858, 0.0728, 0.1526, 0.0537, 0.0227, 0.0252]